In [45]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import tensorflow as tf
from typing import Tuple

In [46]:
def get_data(batch_size: int) -> Tuple[tf.data.Dataset, tf.data.Dataset]:
  (training_images, training_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

  train_dataset = tf.data.Dataset.from_tensor_slices((training_images, training_labels))
  test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

  train_dataset = train_dataset.map(lambda image, label: (float(image) / 255.0, label))
  test_dataset = test_dataset.map(lambda image, label: (float(image) / 255.0, label))

  batch_size = 64
  train_dataset = train_dataset.batch(batch_size).shuffle(500)
  test_dataset = test_dataset.batch(batch_size).shuffle(500)

  return (train_dataset, test_dataset)

In [47]:
class NeuralNetwork(tf.keras.Model):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.sequence = tf.keras.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(20, activation='relu'),
      tf.keras.layers.Dense(10)
    ])

  def call(self, x: tf.Tensor) -> tf.Tensor:
    y_prime = self.sequence(x)
    return y_prime

In [48]:
learning_rate = 0.1
batch_size = 64

(train_dataset, test_dataset) = get_data(batch_size)

model = NeuralNetwork()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate)
metrics = ['accuracy']
model.compile(optimizer, loss_fn, metrics)

In [49]:
epochs = 5
print('\nFitting:')
model.fit(train_dataset, epochs=epochs)


Fitting:
Epoch 1/5


938/938 [==============================] - 2s 1ms/step - loss: 0.6215 - accuracy: 0.7829
Epoch 2/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4521 - accuracy: 0.8394
Epoch 3/5
938/938 [==============================] - 2s 1ms/step - loss: 0.4160 - accuracy: 0.8524
Epoch 4/5
938/938 [==============================] - 2s 1ms/step - loss: 0.3963 - accuracy: 0.8586
Epoch 5/5
938/938 [==============================] - 2s 1ms/step - loss: 0.3795 - accuracy: 0.8638


In [50]:
print('\nEvaluating:')
(test_loss, test_accuracy) = model.evaluate(test_dataset)
print(f'\nTest accuracy: {test_accuracy * 100:>0.1f}%, test loss: {test_loss:>8f}')


Evaluating:


157/157 [==============================] - 0s 922us/step - loss: 0.4131 - accuracy: 0.8514

Test accuracy: 85.1%, test loss: 0.413080


In [51]:
model.save('outputs/model')

INFO:tensorflow:Assets written to: outputs/model/assets


INFO:tensorflow:Assets written to: outputs/model/assets
